In [1]:
from src.incoming_wip.incoming_wip_generator import IncomingWipGenerator
from src.incoming_wip.incoming_wip_config import default_incoming_wip_config
import scipy
from scipy.stats import gamma
import numpy as np
import pandas as pd
import plotly
import plotly.express as px

from tank_state import default_tank_state
from model_config import ModelConfig
import numpy as np
import pandas as pd


In [ ]:

x_values = np.linspace(0, 10, 100)
y_values = gamma.pdf(x=x_values, a=3, scale=1, loc=5)
gamma_df = pd.DataFrame({'x': x_values, 'y': y_values})

fig = px.scatter(data_frame=gamma_df, x='x', y='y')
fig.show()

In [ ]:

for rand_seed in range(5):
    test_config = ModelConfig(
        num_tanks=3,
        tank_status=[default_tank_state] * 3,
        tank_fill_capacity_kg=200,
        tank_min_capacity_kg=60,
        min_wip_cluster=1,
        max_wip_cluster=4,
        max_wip_magnitude=200,
        wafer_slurry_consumption_kg=3,
        tank_clean_minutes=60,
        tank_fill_minutes=90,
        lookahead_minutes=60 * 8,
        bin_minutes=10,
        rand_seed=rand_seed,
    )
    incoming_wip = IncomingWipGenerator(test_config).generate()
    incoming_wip_time = np.arange(0, len(incoming_wip))
    plot_incoming_wip_df = pd.DataFrame({'x': incoming_wip_time, 'y': incoming_wip})
    fig = px.scatter(plot_incoming_wip_df, x='x', y='y')
    fig.show()


In [2]:
incoming_wip = IncomingWipGenerator(default_incoming_wip_config).generate()
incoming_wip

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.06439906,
        0.82227114,  2.0395552 ,  3.37621369,  4.63026888,  5.6929343 ,
        6.51667366,  7.09273615,  7.43559081,  7.57234382,  7.53572087,
        7.35957185,  7.07613506,  6.7145081 ,  6.29992726,  5.85357227,
        5.39269738,  4.93095107,  5.0483021 ,  5.97892995,  7.21952382,
        8.46288212,  9.53604505, 10.35509014, 10.89323147, 11.15862033,
       11.17916625, 10.99239315, 10.63886782, 10.15813034,  9.58634797,
        8.95513189,  8.29111631,  7.61601799,  6.94697992,  6.29706574,
        5.67581605,  5.08980968,  4.54319517,  4.03817324,  3.57542114,
        3.15445665,  2.77394374,  2.4319443 ])

In [1]:
from src.dp_model.dp_model_config import DPModelConfig
import numpy as np
import pandas as pd
from collections import namedtuple

In [6]:
from src.scenario.scenario_config import default_scenario_config

default_scenario_config

ScenarioConfig(lookahead_minutes=480, period_bin_minutes=10, slurry_unit_bin_kg=1, lookahead_period=48)

In [4]:
import numpy as np
from src.dp_model.tank_state import TankState
import operator
test_state = [TankState(0, 100, 0, 0, 3), TankState(1, 100, 0, 0, 1), TankState(2, 100, 0, 0, np.inf)]
min(test_state, key=operator.attrgetter('down_start_period'))

TankState(id=1, current_slurry_unit=100, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=1)

In [3]:
from src.dp_model.dp_model_config import default_dp_model_config
from src.dp_model.dp_model import DpModel

dp_model = DpModel(default_dp_model_config)
dp_model.run(incoming_wip)

In [4]:
dp_model.get_best_end_state()

(State(period=48, active_tank=1, tank_states=(TankState(id=0, current_slurry_unit=200, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf), TankState(id=1, current_slurry_unit=112.0, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf), TankState(id=2, current_slurry_unit=200, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf))),
 StateValue(cost=0, origin=State(period=47, active_tank=1, tank_states=(TankState(id=0, current_slurry_unit=200, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf), TankState(id=1, current_slurry_unit=114.0, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf), TankState(id=2, current_slurry_unit=200, cleaning_period_remaining=0, filling_period_remaining=0, down_start_period=inf)))))

In [4]:
import mip